<a href="https://colab.research.google.com/github/khamkarajinkya/Recommender-Practice/blob/main/gale-shapley-stable-matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import io 

In [2]:
e_pref = '''
Employee,Programmer,Manager,Marketing,Mechanic,Post-Man,Supervisor
Pritz-Brown,3,5,2,1,6,4
Sheryl,4,5,3,2,5,1
Robin-Hood,1,2,3,5,6,4
Charles,5,2,3,4,1,6
Andysah,2,5,3,6,4,1
Keysha,6,1,4,5,3,2
'''

c_pref = '''
Company,emp1,emp2,emp3,emp4,emp5,emp6
Programmer,1,2,4,5,3,6
Manager,5,4,2,3,1,6
Marketing,4,1,2,5,6,3
Mechanic,3,1,6,5,2,4
Post-Man,1,6,4,5,3,2
Supervisor,2,3,4,6,5,1
'''

In [3]:
e_pref = pd.read_csv(io.StringIO(e_pref))
c_pref = pd.read_csv(io.StringIO(c_pref)).set_index('Company').T

In [4]:
e_dict = e_pref.set_index('Employee').T.to_dict('list')
c_dict = c_pref.T.to_dict('list')

In [5]:
pref_score = {}

for n, v in e_dict.items():
  pref_score[n] = {}
  for e, k in c_dict.items():
    run_denom = 0
    for i in range(len(v)):
      run_denom += 1 / (v[i] + k[i])
    pref_score[n][e] = len(v) / run_denom
  

In [6]:
'''
Gale Shapley Algorithm

Initialize all m ∈ M and w ∈ W to free
  while ∃ m who still has a w to select  {
      w = first w on m’s list to whom m has not yet selected
      if w is free:
        (m, w) paired
      else some pair (m', w) already exists
      if w prefers m to m'
        m' becomes free
        (m, w) become paired
      else
        (m', w) remain paired

'''
optimal = {}
cmp_score = {}
cmp_pair = {}
emp_avl = list(e_dict.keys())
cmp_avl = list(c_dict.keys())

while emp_avl and cmp_avl:
  for e in emp_avl:
    c = max(pref_score[e], key=pref_score[e].get)
    #check if prefered company is available 
    if c in cmp_avl:
      '''
      1. remove company from list of available
      2. add employer e preference as c
      3. Set company score c to hs
      4. Track current cmp pair for company c to e
      '''
      cmp_avl.remove(c)
      optimal[e] = c
      cmp_score[c]  = pref_score[e][c]
      cmp_pair[c] = e
      emp_avl.remove(e)
    else:
      #check if employee desirable to company
      if cmp_score[c] < pref_score[e][c]:
        '''
        1. update company score to hs
        2. add company c pair back to employee list
        3. remove company from available list
        4. append old company to available list
        5. update optimal list for current employee
        '''
        optimal.pop(cmp_pair[c])
        cmp_score[c] = pref_score[e][c]
        emp_avl.append(cmp_pair[c])
        optimal[e] = c
        emp_avl.remove(e)
    print(f'{e} available {list(pref_score[e].keys())} and max {c} with score {pref_score[e][c]} and optimal {optimal}')
    pref_score[e].pop(c, None)
  print('\n\n')

Pritz-Brown available ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6'] and max emp4 with score 7.99538505912893 and optimal {'Pritz-Brown': 'emp4'}
Robin-Hood available ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6'] and max emp4 with score 7.666989351403679 and optimal {'Pritz-Brown': 'emp4'}
Charles available ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6'] and max emp4 with score 7.632508833922261 and optimal {'Pritz-Brown': 'emp4'}
Andysah available ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6'] and max emp4 with score 8.133007334963327 and optimal {'Andysah': 'emp4'}
Pritz-Brown available ['emp1', 'emp2', 'emp3', 'emp5', 'emp6'] and max emp3 with score 6.640316205533597 and optimal {'Andysah': 'emp4', 'Pritz-Brown': 'emp3'}



Sheryl available ['emp1', 'emp2', 'emp3', 'emp4', 'emp5', 'emp6'] and max emp4 with score 8.034006376195537 and optimal {'Andysah': 'emp4', 'Pritz-Brown': 'emp3'}
Robin-Hood available ['emp1', 'emp2', 'emp3', 'emp5', 'emp6'] and max emp6 with score 6.8915

In [7]:
optimal

{'Andysah': 'emp4',
 'Charles': 'emp1',
 'Keysha': 'emp2',
 'Pritz-Brown': 'emp3',
 'Robin-Hood': 'emp6',
 'Sheryl': 'emp5'}